### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
            self.tree = dict()
            self.min_samples_split = min_samples_split
            self.max_depth = max_depth
            self.sufficient_share = sufficient_share
            self.num_class = -1
            self.feature_importances_ = None
            if criterion == 'gini':
                self.G_function = self.__gini
            elif criterion == 'entropy':
                self.G_function = self.__entropy
            elif criterion == 'misclass':
                self.G_function = self.__misclass
            else:
                print('invalid criterion name')
                raise

            if max_features == 'sqrt':
                self.get_feature_ids = self.__get_feature_ids_sqrt
            elif max_features == 'log2':
                self.get_feature_ids = self.__get_feature_ids_log2
            elif max_features is None:
                self.get_feature_ids = self.__get_feature_ids_N
            else:
                print('invalid max_features name')
                raise


    def __gini(self, l_c, l_s, r_c, r_s):
        '''
        If a data set D is split into D1 and D2 then
        gini index is computed as follows:
        gini(D) = |D1|/|D| * gini(D1) + |D2|/|D| * gini(D2)
        '''
        return 1 - np.sum((l_c**2 / l_s + r_c**2 / r_s) / (l_s+r_s), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c * np.log2((l_c + 1e-4) / l_s) + r_c *
                       np.log2((r_c + 1e-4) / r_s),
                       axis=1).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(
            l_c / (l_s + r_s), axis=1) - np.max(r_c / (l_s + r_s),
                                                axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.random.permutation(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
            '''
            1. Sort the values of a feature _id {v_1, v_2, ...}
            2. Consider split (v_i + v_{i+1}) / 2
            3. Compute impurities for threshold for each i
            4. Choose res_thr with the smallest error
            '''
            feature_ids = self.get_feature_ids(x.shape[1])
            _min = np.inf
            for _id in feature_ids:
                x_sorted, y_sorted = self.__sort_samples(x[:, _id], y)
                borders = np.where(y_sorted[:-1] - y_sorted[1:] != 0)[0] + 1
                lens = (borders - np.hstack((np.array([0]),
                        borders[:-1]))).reshape(-1, 1)
                one_hot = np.zeros((borders.shape[0], self.num_class))
                one_hot[np.arange(borders.shape[0]), y_sorted[borders - 1]] = 1
                one_hot *= lens
                l_c = np.cumsum(one_hot, axis=0).astype(int)
                r_c = np.bincount(y_sorted, minlength=self.num_class) - l_c
                l_s = np.sum(l_c, axis=1).reshape(-1, 1).astype(int)
                r_s = y_sorted.shape[0] - l_s
                gs = self.G_function(l_c, l_s, r_c, r_s)
                ind = np.argmin(gs)
                left_ind = l_s[ind][0]
                if gs[ind] < _min:
                    res_ind = _id
                    res_thr = (x_sorted[left_ind - 1] + x_sorted[left_ind]) / 2
                    _min = gs[ind]
            '''
            1. initialize an array feature_importances of all zeros with
            size n_features
            2. traverse the tree: for each internal node that splits on feature
            i compute the error reduction of that node multiplied by the number
            of samples that were routed to the node and add this quantity to
            feature_importances[i]
            The error reduction depends on the impurity criterion that use
            (e.g. Gini, Entropy, MSE, ...)
            Its the impurity of the set of examples that gets routed to the
            internal node minus the sum of the impurities of the two partitions
            created by the split.
            '''
            reduction = self.G_function(np.expand_dims(np.bincount(y), axis=0),
                                        y.shape[0], 0, 1e-4) - _min
            self.feature_importances_[res_ind] += x.shape[0] * reduction
            return res_ind, res_thr
        
    def __fit_node(self, x, y, node_id, depth):
        x = np.asarray(x).astype(float)
        y = np.asarray(y).astype(int)
        if (depth == self.max_depth or
                self.min_samples_split >= x.shape[0] or
                np.unique(y).shape[0] == 1):
            cnt = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE,
                                  cnt.argmax(), cnt / y.shape[0])
            return
        feature_id, threshold = self.__find_threshold(x, y)
        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE,
                              feature_id, threshold)
        X_l, X_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)
        if X_l.shape[0] == 0 or X_r.shape[0] == 0:
            cnt = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, cnt.argmax(),
                                  cnt.astype(float) / y.shape[0])
            return
        self.__fit_node(X_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(X_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1]).astype(float)
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= x.shape[0]

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)


In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 14.9 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8885003885003885

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
                 'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
                  'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o', 'field', 'undergra',
                  'imprelig', 'imprace',
                  'from', 'zipcode', 'career', 'sports',
                  'tvsports', 'exercise',
                  'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
pd.get_dummies(df, columns=['race'], prefix='race', prefix_sep='=')
df.loc[:, 'mn_sat'] =\
df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] =\
df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df.loc[:, 'income'] =\
df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] =\
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                   'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
               'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                         'fun1_1', 'amb1_1', 'shar1_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']]\
                                   .sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
               'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                         'fun2_1', 'amb2_1', 'shar2_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [11]:
my_clf = MyDecisionTreeClassifier()
clf = DecisionTreeClassifier()

## Проверка скорости работы на Speed Dating Data 

In [14]:
%time my_clf.fit(X_train, y_train)

Wall time: 549 ms


In [15]:
%time clf.fit(X_train, y_train)

Wall time: 65.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Проверка качества работы на Speed Dating Data

In [16]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5403472931562819

In [17]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.581407166528659

## Задание 3

In [18]:
ind = np.argsort(clf.feature_importances_)
print(*zip(df_pair.columns[ind][:-10:-1], clf.feature_importances_[ind][:-10:-1]), sep='\n')

('match', 0.05918533895001461)
('fun1_1_f', 0.03335807334834059)
('goal_f', 0.03249111237125304)
('amb3_1', 0.031182624565367335)
('attr1_1_f', 0.027341450416352518)
('fun2_1', 0.026407149755228898)
('exphappy_f', 0.026372475390946136)
('amb2_1_f', 0.025625202777600806)
('attr2_1_f', 0.02486489777950578)


In [19]:
ind = np.argsort(my_clf.feature_importances_)
print(*zip(df_pair.columns[ind][:-10:-1], my_clf.feature_importances_[ind][:-10:-1]), sep='\n')

('date_f', 0.010487924310392898)
('career_c_f', 0.007785845272846125)
('goal', 0.007691400039093406)
('go_out', 0.007053011855848473)
('race_f', 0.006409640825736037)
('int_corr', 0.005790464601308963)
('fun3_1_f', 0.005472086016623746)
('income_f', 0.00463972556362284)
('intel3_1', 0.0045840962116136755)


## Задание 4

## Задание 5

In [20]:
params = {
    "max_depth": np.linspace(1, 10, 10).astype(int),
    "min_samples_split": np.linspace(2, 10, 8).astype(int),
    "criterion": ["gini", "entropy"],
}
rf = GridSearchCV(RandomForestClassifier(n_estimators=10), params, cv=3)


In [21]:
rf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8, 10]), 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
rf.best_params_

{'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 7}